In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format = 'retina'

# DataMap examples

## PVDATA

Get some actual data that we will use to map

In [2]:
import json
import os

In [3]:
rdat = json.load(open('data/live_model_PVDB.json'))

PVDATA = {k:rdat[k]['value'] for k in rdat}



# Tabular

Often PVs have a simple linear mapping to simulation inputs. The `TabularDataMap` helps with this

In [4]:
from lcls_live.datamaps.tabular import TabularDataMap
from lcls_live import data_dir

import pandas as pd
import dataclasses 

In [5]:
# Make some tabular data

dat = [
    
    {'name': 'BC1_offset',
     'pvname':'BMLN:LI21:235:MOTR',  # mm
     'bmad_factor': 0.001,
     'bmad_name': 'O_BC1_OFFSET',
     'bmad_attribute': 'offset'
    },
    
    {'name': 'BC2_offset',
     'pvname':'BMLN:LI24:805:MOTR', # mm
     'bmad_factor': 0.001,
     'bmad_name': 'O_BC2_OFFSET',
     'bmad_attribute': 'offset'
    },
    
    {
    'name': 'L2_phase',
    'pvname': 'ACCL:LI22:1:PDES',
    'bmad_name':'O_L2',
    'bmad_factor': 1,
    'bmad_attribute':'phase_deg'
    
},
    {
    'name': 'L3_phase',
    'pvname': 'ACCL:LI25:1:PDES',
    'bmad_name':'O_L3',
    'bmad_attribute':'phase_deg',
    'bmad_offset': 0
    
},
]

#Note that there are sone NaNs here. That's okay.
df = pd.DataFrame(dat)
df

,name,pvname,bmad_factor,bmad_name,bmad_attribute,bmad_offset
0,BC1_offset,BMLN:LI21:235:MOTR,0.001,O_BC1_OFFSET,offset,NaN
1,BC2_offset,BMLN:LI24:805:MOTR,0.001,O_BC2_OFFSET,offset,NaN
2,L2_phase,ACCL:LI22:1:PDES,1.000,O_L2,phase_deg,NaN
3,L3_phase,ACCL:LI25:1:PDES,NaN,O_L3,phase_deg,0.0


In [6]:
# Make the DataMap object, identifying the columns to be used
DM = TabularDataMap(df, pvname='pvname', element='bmad_name', attribute='bmad_attribute', factor='bmad_factor', offset='bmad_offset')

DM.pvlist

['BMLN:LI21:235:MOTR',
 'BMLN:LI24:805:MOTR',
 'ACCL:LI22:1:PDES',
 'ACCL:LI25:1:PDES']

In [7]:
# Process the data for Bmad commands
DM.as_bmad(PVDATA)

['O_BC1_OFFSET[offset] = 0.001 * 247.85581047127175',
 'O_BC2_OFFSET[offset] = 0.001 * 385.0',
 'O_L2[phase_deg] = -35.882813702404',
 'O_L3[phase_deg] = 0.0']

In [8]:
# or Tao commands
DM.as_tao(PVDATA)

['set ele O_BC1_OFFSET offset = 0.001 * 247.85581047127175',
 'set ele O_BC2_OFFSET offset = 0.001 * 385.0',
 'set ele O_L2 phase_deg = -35.882813702404',
 'set ele O_L3 phase_deg = 0.0']

In [9]:
# Save, and reload
JSON_OUT = os.path.join(data_dir, 'cu/linac_TabularDataMap.json')

DM.to_json(JSON_OUT)

DM2 = TabularDataMap.from_json(open(JSON_OUT).read())
DM2.data

,name,pvname,bmad_factor,bmad_name,bmad_attribute,bmad_offset
0,BC1_offset,BMLN:LI21:235:MOTR,0.001,O_BC1_OFFSET,offset,NaN
1,BC2_offset,BMLN:LI24:805:MOTR,0.001,O_BC2_OFFSET,offset,NaN
2,L2_phase,ACCL:LI22:1:PDES,1.000,O_L2,phase_deg,NaN
3,L3_phase,ACCL:LI25:1:PDES,NaN,O_L3,phase_deg,0.0


In [10]:
# Read a previously made csv file. This has slightly different columns
df2 = pd.read_csv('../lcls_live/data/cu_hxr/quad_mapping.csv')[0:10]
df2.columns

Index(['device_name', 'attribute', 'unit', 'bmad_ele_name', 'bmad_factor',
       'bmad_attribute', 'example_value'],
      dtype='object')

In [11]:
df2['pvname'] = df2['device_name'] + ':' + df2['attribute']

DM2 = TabularDataMap(df2, pvname='pvname', element='bmad_ele_name', attribute='bmad_attribute', factor='bmad_factor', offset='')
DM2.pvlist

['QUAD:LI21:211:BDES',
 'QUAD:LI21:221:BDES',
 'QUAD:LI21:243:BDES',
 'QUAD:LI21:251:BDES',
 'QUAD:LI21:271:BDES',
 'QUAD:LI21:335:BDES',
 'QUAD:LI24:713:BDES',
 'QUAD:LI24:740:BDES',
 'QUAD:LI24:860:BDES',
 'QUAD:LI24:892:BDES']

In [12]:
# Here these aren't in our PVDATA
DM2.as_tao(PVDATA)

['! Bad value for QM11[b1_gradient]: None',
 '! Bad value for CQ11[b1_gradient]: None',
 '! Bad value for SQ13[b1_gradient]: None',
 '! Bad value for CQ12[b1_gradient]: None',
 '! Bad value for QM12[b1_gradient]: None',
 '! Bad value for QM15[b1_gradient]: None',
 '! Bad value for QM21[b1_gradient]: None',
 '! Bad value for CQ21[b1_gradient]: None',
 '! Bad value for CQ22[b1_gradient]: None',
 '! Bad value for QM22[b1_gradient]: None']

In [13]:
# We could  check beforehand:
missing = [name for name in DM2.pvlist if name not in PVDATA]
len(missing)

10

# Klystrons

In [14]:
from lcls_live.datamaps.klystron import KlystronDataMap, klystron_pvinfo, existing_LCLS_klystrons_sector_station

In [15]:
# Get a sector, station that exists
existing_LCLS_klystrons_sector_station[0]

(20, 6)

In [16]:
# This will return a flat dict of info
klystron_pvinfo(20, 6)

{'name': 'K20_6',
 'sector': 20,
 'station': 6,
 'description': 'Klystron in sector 20, station 6 for the GUN',
 'enld_pvname': 'GUN:IN20:1:GN1_AAVG',
 'phase_pvname': 'GUN:IN20:1:GN1_PAVG'}

In [17]:
?KlystronDataMap

Init signature:
KlystronDataMap(
    name: str,
    sector: int,
    station: int,
    description: str = '',
    enld_pvname: str = '',
    phase_pvname: str = '',
    accelerate_pvname: str = '',
    swrd_pvname: str = '',
    stat_pvname: str = '',
    hdsc_pvname: str = '',
    dsta_pvname: str = '',
) -> None
Docstring:     
Attributes
----------
bmad_name : str
pvlist : list[str]

Methods
-------
evaluate(pvdata) :
    Returns
    -------
    dict of:
        enld : float
            energy gain in MeV
        phase : float
            phase in deg
        in_use : bool
            
            
    

as_bmad(pvdata)
File:           ~/Code/GitHub/lcls-live/lcls_live/datamaps/klystron.py
Type:           type
Subclasses:     


In [18]:
# This makes an object
KlystronDataMap(**klystron_pvinfo(20, 6))

KlystronDataMap(name='K20_6', sector=20, station=6, description='Klystron in sector 20, station 6 for the GUN', enld_pvname='GUN:IN20:1:GN1_AAVG', phase_pvname='GUN:IN20:1:GN1_PAVG', accelerate_pvname='', swrd_pvname='', stat_pvname='', hdsc_pvname='', dsta_pvname='')

In [19]:
k = KlystronDataMap(**klystron_pvinfo(20, 6))

# These are the PV names needed to mapping data
k.pvlist


['GUN:IN20:1:GN1_AAVG', 'GUN:IN20:1:GN1_PAVG']

In [20]:
# This will extract those and produce useful information
k.evaluate(PVDATA)

{'enld': 0.000929853773306308, 'phase': 3.8561623144726838, 'in_use': True}

In [21]:
# Actual inputs for a simulation
k.as_bmad(PVDATA)

['O_K20_6[ENLD_MeV] = 0.000929853773306308',
 'O_K20_6[phase_deg] = 3.8561623144726838',
 'O_K20_6[in_use] = 1']

In [22]:
# Make a large dict to look up a DataMap for any existing klystron by name

KLYSTRON_DATAMAP = {}
for sector, station in existing_LCLS_klystrons_sector_station:
   # print(sector, station)    
    info = klystron_pvinfo(sector, station)
    k = KlystronDataMap(**info)

    KLYSTRON_DATAMAP[info['name']] = k

In [23]:
# Check that our data is sufficient 

for name, k in KLYSTRON_DATAMAP.items():
    for pv in k.pvlist:
        if pv not in PVDATA:
            print(k.name, pv)

## Export to JSON

Using the `dataclasses.asdict` function, the object can be endoded for dumping to JSON, and then simply reloaded.

In [24]:
import dataclasses
class EnhancedJSONEncoder(json.JSONEncoder):
    def default(self, o):
        if dataclasses.is_dataclass(o):
            return dataclasses.asdict(o)
        return super().default(o)

# Example serialization
json.dumps(KLYSTRON_DATAMAP['K20_6'], cls=EnhancedJSONEncoder)

'{"name": "K20_6", "sector": 20, "station": 6, "description": "Klystron in sector 20, station 6 for the GUN", "enld_pvname": "GUN:IN20:1:GN1_AAVG", "phase_pvname": "GUN:IN20:1:GN1_PAVG", "accelerate_pvname": "", "swrd_pvname": "", "stat_pvname": "", "hdsc_pvname": "", "dsta_pvname": ""}'

In [25]:
# Export to file and reload

JSONFILE = os.path.join(data_dir, 'cu/klystron_datamaps.json')

json.dump(KLYSTRON_DATAMAP, open(JSONFILE, 'w'), cls=EnhancedJSONEncoder)

KLYSTRON_DATAMAP2 = {k:KlystronDataMap(**v) for k, v in json.load(open(JSONFILE)).items()}
KLYSTRON_DATAMAP2['K20_6']

KlystronDataMap(name='K20_6', sector=20, station=6, description='Klystron in sector 20, station 6 for the GUN', enld_pvname='GUN:IN20:1:GN1_AAVG', phase_pvname='GUN:IN20:1:GN1_PAVG', accelerate_pvname='', swrd_pvname='', stat_pvname='', hdsc_pvname='', dsta_pvname='')